<a href="https://colab.research.google.com/github/RogerHeederer/NLPwithPyTorch_book/blob/main/SentimentClassfi_Fullyelp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Minimally clean the data and derive final dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import collections
import numpy as np
import pandas as pd
import re

from argparse import Namespace

In [4]:
args = Namespace(
    raw_train_dataset_csv = "/content/drive/MyDrive/RogerHeederer/NLPwithPytorch_B/data/dataset/yelp/raw_train.csv",
    raw_test_dataset_csv = "/content/drive/MyDrive/RogerHeederer/NLPwithPytorch_B/data/dataset/yelp/raw_test.csv",
    proportion_subset_of_train = 0.1,
    train_proportion = 0.7,
    val_proportion=0.15,
    test_proportion=0.15,
    output_munged_csv="/content/drive/MyDrive/RogerHeederer/NLPwithPytorch_B/data/dataset/yelp/reviews_with_splits_full.csv",
    seed=1337
)

데이터 성격 파악 및 조작법

In [5]:
train_reviews = pd.read_csv(args.raw_train_dataset_csv, header=None, names=['rating', 'review'])

In [6]:
train_reviews[1:10]

,rating,review
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...
5,1,Wing sauce is like water. Pretty much a lot of...
6,1,Owning a driving range inside the city limits ...
7,1,This place is absolute garbage... Half of the...
8,2,Before I finally made it over to this range I ...
9,2,I drove by yesterday to get a sneak peak. It ...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
train_reviews.shape

(560000, 2)

In [8]:
train_reviews_test = pd.read_csv(args.raw_train_dataset_csv, header=None, names=['rating', 'review'])

In [9]:
train_reviews_test[0:2]['review']

0    Unfortunately, the frustration of being Dr. Go...
1    Been going to Dr. Goldberg for over 10 years. ...
Name: review, dtype: object

In [10]:
#위 표현과 같은 것
train_reviews_test.review[0:2]

0    Unfortunately, the frustration of being Dr. Go...
1    Been going to Dr. Goldberg for over 10 years. ...
Name: review, dtype: object

In [11]:
train_reviews_test[0:2]['review'][0] = np.nan
train_reviews_test[0:2]['review'][1] = ''

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
train_reviews_test[0:2]['review']

0    NaN
1       
Name: review, dtype: object

In [13]:
pd.isnull(train_reviews_test[0:2]['review'])

0     True
1    False
Name: review, dtype: bool

In [14]:
~pd.isnull(train_reviews_test.review[0:2])

0    False
1     True
Name: review, dtype: bool

데이터 로드 - 결측치를 제거 시키고 다운로드 한다

In [15]:
train_reviews = pd.read_csv(args.raw_train_dataset_csv, header=None, names=['rating', 'review'])
train_reviews = train_reviews[~pd.isnull(train_reviews.review)] #nan이 아닌 것만 train_reivews로 가져온다. 즉 결측치 제거
test_reviews = pd.read_csv(args.raw_test_dataset_csv, header=None, names=['rating', 'review'])
test_reviews = test_reviews[~pd.isnull(test_reviews.review)] # nan 제거하고 담음 결측치 제거

In [16]:
train_reviews.head()

,rating,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [51]:
test_reviews.head()

,rating,review
0,1,Ordered a large Mango-Pineapple smoothie. Stay...
1,2,Quite a surprise! \n\nMy wife and I loved thi...
2,1,"First I will say, this is a nice atmosphere an..."
3,2,I was overall pretty impressed by this hotel. ...
4,1,Video link at bottom review. Worst service I h...


In [17]:
set(train_reviews.rating)

{1, 2}

In [18]:
# dict 만들기
by_rating = collections.defaultdict(list)
for _, row in train_reviews.iterrows():
    by_rating[row.rating].append(row.to_dict()) # rating값이 key.

In [19]:
by_rating.keys()

dict_keys([1, 2])

In [24]:
#data split
final_list = []
np.random.seed(args.seed)

for _, item_list in sorted(by_rating.items()):
    np.random.shuffle(item_list)

    n_total = len(item_list)
    n_train = int(args.train_proportion * n_total)
    n_val = int(args.val_proportion * n_total)
    
    #기존 데이터에 새로운 특징 추가
    for item in item_list[:n_train]: #훈련 데이터 셋 붙이기
        item['split'] = 'train'
    
    for item in item_list[n_train:n_train+n_val]: #검증 데이터 셋 붙이기/ 테스트는 따로 붙임
        item['split'] = 'val'

    final_list.extend(item_list)

In [25]:
final_list[9999]

{'rating': 1,
 'review': "The customer service is so bad here. You walk in and you're not greeted. I've had 3 separate occasions to be here and never once was I greeted. No one looks up to speak. When you finally do catch someones attention they simply tell you to sign in. They may or may not be working with a customer. You sign in, then they acknowledge you, even if no one else is in the store!! \\nThe only reason they do not have 1 star is because they have a large variety of frames and the prices for frames are reasonable. As far as getting your lenses I felt those prices were high even for their basic lens without any upgrades. I don't have vision insurance so eyeglass world it is but it is definitely not my first pick and wouldn't recommend them to anyone I like.",
 'split': 'train'}

## dict itteration 도는 부분이 헷갈려서 세부적으로 나눠 정리해봄. 아래 참조

In [ ]:
example = train_reviews[1:10].copy()
example['review'][1] = np.nan

In [64]:
for _, row in example.iterrows():
    print(_)

1
2
3
4
5
6
7
8
9


In [65]:
for _, row in example.iterrows():
    print(row)

rating      2
review    NaN
Name: 1, dtype: object
rating                                                    1
review    I don't know what Dr. Goldberg was like before...
Name: 2, dtype: object
rating                                                    1
review    I'm writing this review to give you a heads up...
Name: 3, dtype: object
rating                                                    2
review    All the food is great here. But the best thing...
Name: 4, dtype: object
rating                                                    1
review    Wing sauce is like water. Pretty much a lot of...
Name: 5, dtype: object
rating                                                    1
review    Owning a driving range inside the city limits ...
Name: 6, dtype: object
rating                                                    1
review    This place is absolute garbage...  Half of the...
Name: 7, dtype: object
rating                                                    2
review    Before I finally made

In [66]:
for _, row in example.iterrows():
    row_dict = row.to_dict()
    print(row_dict)

{'rating': 2, 'review': nan}
{'rating': 1, 'review': "I don't know what Dr. Goldberg was like before  moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in the co-pay and having you come in for medication refills every month. He will not give refills and could less about patients's financial situations. Trying to get your 90 days mail away pharmacy prescriptions through this guy is a joke. And to make matters even worse, his office staff is incompetent. 90% of the time when you call the office, they'll put you through to a voice mail, that NO ONE ever answers or returns your call. Both my adult children and husband have decided to leave this practice after experiencing such frustration. The entire office has an attitude like they are doing you a favor. Give me a break! Stay away from this doc and the practice. You deserve 

In [59]:
#테스트 데이터 셋을 final_list에 최종 붙이기
for _, row in test_reviews.iterrows():
    row_dict = row.to_dict()
    row_dict['split'] = 'test'
    final_list.append(row_dict)

In [68]:
final_list[0:5]

[{'rating': 1,
  'review': "The entrance was the #1 impressive thing about this place, as it is completely a surprise and almost shocks you.  I won't give it up here but it's worth at least getting a drink to experience that part.\\n\\nGreeter was great.\\n\\nLounge singer was very 70s and campy, but liked him none the less.\\n\\n\\nFood is pretty below average in pretty much every way possible.\\n\\nBread they bring out is pretty bad and dries out within minutes of being at the table to the point it's inedible.\\n\\nSalads were small, boring and WAY drowned in dressing.\\n\\nCalamari is $15 or so, and is half the size of what they'd give you at Capitol Grille, and is pretty bland and mushy.\\n\\nthe pasta tasted as though it were cooked in rancid water.  the chef was nice enough to come out and bring my wife a personalized dish that he eats, and it tasted the same.  we smiled and pretended to enjoy it just to keep from looking like big complainers.  We literally don't complain usually

In [69]:
final_reviews = pd.DataFrame(final_list)

In [74]:
final_reviews[480000:480005]

,rating,review,split
480000,2,Mon Ami Gabi is located in Paris hotel (visibl...,val
480001,2,I love the food here & the lemonade is freshly...,val
480002,2,I love this place. I live in San Diego and the...,val
480003,2,After the tiring walk in the heated sun when i...,val
480004,2,Lunch review: \nGreat quick lunch options. I'...,val


In [70]:
final_reviews.split.value_counts()

train    392000
val       84000
test      38000
Name: split, dtype: int64

In [71]:
final_reviews.review.head()

0    The entrance was the #1 impressive thing about...
1    I'm a Mclover, and I had no problem\nwith the ...
2    Less than good here, not terrible, but I see n...
3    I don't know if I can ever bring myself to go ...
4    Food was OK/Good but the service was terrible....
Name: review, dtype: object

In [76]:
final_reviews[pd.isnull(final_reviews.review)] #null값 검사해서 true인 위치 가져와서 뿌려줘

,rating,review,split


In [77]:

# Preprocess the reviews
def preprocess_text(text):
    if type(text) == float:
        print(text)
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text
    
final_reviews.review = final_reviews.review.apply(preprocess_text)

In [78]:
final_reviews['rating'] = final_reviews.rating.apply({1: 'negative', 2: 'positive'}.get)

In [79]:
final_reviews.head()

,rating,review,split
0,negative,the entrance was the impressive thing about th...,train
1,negative,"i m a mclover , and i had no problem nwith the...",train
2,negative,"less than good here , not terrible , but i see...",train
3,negative,i don t know if i can ever bring myself to go ...,train
4,negative,food was ok good but the service was terrible ...,train
